In [ ]:
import pandas as pd
import time
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
vertex_project = os.environ['VERTEX_PROJECT_ID']

In [ ]:
def generate(prompt):
  vertexai.init(project=vertex_project, location="us-central1")
  model = GenerativeModel("gemini-1.0-pro-002")
  responses = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 200,
        "temperature": 0.8,
        "top_p": 1
    },
    safety_settings={
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
    stream=False,
  )
  
  return responses

In [ ]:
df = pd.read_csv("")


print(len(df))
print()
df.head()

In [ ]:
def construct_prompt(titulo):
    resp = f'''### Instruções:
    Você receberá o título de um artigo. Você deve extrair o nome da empresa presente no título e o código da empresa, quando disponíveis. Caso não esteja presente no título, responda com NA.
    
    ### Exemplo 1
    Título: Braskem (BRKM5) | Os riscos de Maceió
    
    Resposta:
    (Brasken, BRKM5)
    
    ### Exemplo 2
    Título: Braskem | Os riscos de Maceió
    
    Resposta:
    (Brasken, NA)
    
    ### Formato de Saída:
    (empresa, código)
    
    Responda apenas com (empresa, código).
    
    ### Título
    {titulo}'''
    
    return resp

In [ ]:
answers = ["none"] * len(df)
empresas = ["none"] * len(df)
codigos = ["none"] * len(df)

In [ ]:
import re
import unicodedata

def remover_acentos(texto):
    texto_normalizado = unicodedata.normalize('NFD', texto)
    return ''.join([c for c in texto_normalizado if not unicodedata.combining(c)])

def extrair_resposta(texto):
    texto = remover_acentos(texto)
    # Compila as expressões regulares para encontrar a empresa e o código
    texto_limpo = texto.strip().strip('()')
    
    # Divide o texto pela vírgula
    partes = texto_limpo.split(',')
    
    # Extrai a empresa e o código
    nome_empresa = partes[0].strip()
    codigo_empresa = partes[1].strip()
    
    return nome_empresa, codigo_empresa

In [ ]:
i = 0

while i < len(df):
  try:
    prompt = construct_prompt(df["title"].iloc[i])

    resp = generate(prompt)

    resp_texto = resp.text.replace("\n", '')
    answers[i] = resp_texto.lower()
    emp, cod = extrair_resposta(resp_texto)
    
    empresas[i] = emp
    codigos[i] = cod
    
    i += 1

    if i % 50 == 0:
        print("Progresso: ", i)

    # Pausa de 2 segundos (sugerido é 1 solicitação a cada 5s)
    time.sleep(2)

  except Exception as e:
    erro = f"Erro encontrado {i}: {e}. Tentando novamente..."
    print(erro)
    
    answers[i] = erro
    empresas[i] = "NA"
    codigos[i] = "NA"
    
    i += 1
    
    time.sleep(8)  # Pausa antes de tentar novamente

In [ ]:
new_df = df.copy()

In [ ]:
new_df["company"] = empresas
new_df["cod"] = codigos

In [ ]:
new_df.head()

In [ ]:
new_df.to_csv("fr_xp_companies.csv", index=False)